# CARLA Python API - Lab3 (With Solutions)


Lab goals:
- reuse helper functions for faster experiments;
- follow vehicles with the spectator camera;
- inspect waypoints and road topology with debug drawing;
- compare repositioning (`set_transform`) with real driving control (`apply_control`).

## Optional troubleshooting: Conda in VS Code notebooks

If your notebook kernel cannot import `carla`, use this quick check.

### Quick check

In a VS Code Python notebook, click `Select Kernel` (top-right), then run:

```python
import carla
```

Install `ipykernel` if VS Code asks for it.

![Python notebook kernel selection](img/python-kernel.png)

### Conda works in terminal but not in VS Code

1. Find the absolute path of `conda.exe` in terminal (example: `C:\anaconda3\Scripts\conda.exe`).
2. Open VS Code settings (`Ctrl + ,`).
3. Search for `python.condaPath`.
4. Paste the path in `Python: Conda Path`.
   ![Python conda path](img/python-conda-path.png)
5. Restart VS Code and reselect the kernel.

In [ ]:
import carla, time, pygame, random

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## 1. Helper functions used in this notebook

This lab reuses three utility functions:
- `move_spectator_to(...)`: place the camera behind/above a target transform.
- `spawn_vehicle(...)`: spawn a vehicle from the blueprint library at a chosen spawn point.
- `draw_on_screen(...)`: convenience wrapper around `world.debug.draw_string(...)`.

Keep these helpers stable and focus your edits on behavior logic in later sections.

In [4]:
def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(world, vehicle_index=0, spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content="O", color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)


## 2. Make the spectator follow an autonomous vehicle

In this section you spawn a vehicle, enable autopilot, and keep the spectator attached to it.

### Procedure
1. Spawn a vehicle with `spawn_vehicle(world)`.
2. Enable autopilot with `vehicle.set_autopilot(True)`.
3. In a loop, call `move_spectator_to(vehicle.get_transform(), spectator)`.
4. Advance the simulation with `world.tick()`.

Stop with `KeyboardInterrupt`, then destroy the actor with `vehicle.destroy()`.

In [ ]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

while True:
    move_spectator_to(vehicle.get_transform(), spectator)
    world.tick()

In [ ]:
vehicle.destroy()

## 3. Waypoints

Waypoints represent navigable points on the road network and are useful for local path inspection.

In the next code cell:
1. a vehicle is spawned and moved by autopilot;
2. at fixed intervals, the closest waypoint is sampled;
3. sampled waypoint locations are printed for inspection.

![waypoints](img/waypoints.png)

### Example of waypoint sampling

Waypoints can be computed from actor position with `world.get_map().get_waypoint(location)`.

In [ ]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

waypoints_list = []

for _ in range(10):
    transform = vehicle.get_transform()
    waypoint = world.get_map().get_waypoint(transform.location)
    waypoints_list.append(waypoint)
    world.tick()
    time.sleep(1)

for wp in waypoints_list:
    print(wp.transform.location)

In [ ]:
vehicle.destroy()

### Debug utilities: draw elements on screen

Use `world.debug` to visualize geometry while developing logic.

Main methods:
- `world.debug.draw_arrow`
- `world.debug.draw_box`
- `world.debug.draw_line`
- `world.debug.draw_point`
- `world.debug.draw_string`

Guidelines:
- use short `life_time` values during iteration, so the scene stays readable;
- keep color usage consistent per concept (for example, one color for current waypoint, one for target).

In [164]:
for wp in waypoints_list:
    world.debug.draw_string(
        wp.transform.location,
        'Waypoint Here!',
        color=carla.Color(r=0, g=255, b=0),
        life_time=20.0, # life_time=0 means infinite
    )

### Roads

`world.get_map().get_topology()` returns road segments as waypoint pairs `(start_wp, end_wp)`.

The next cells show how to:
1. pick an initial waypoint;
2. traverse forward with `waypoint.next(distance)`;
3. draw visited points to inspect the road structure.

In [8]:
roads = world.get_map().get_topology()

waypoint = roads[0][0]
draw_on_screen(world, waypoint.transform)

In [ ]:
for i in range(1000):
    # get the next waypoint with distance 1
    waypoint = waypoint.next(1)[0]
    draw_on_screen(world, waypoint.transform)
    time.sleep(0.01)

Now we draw multiple roads with different colors to make segment boundaries easier to read.

In [14]:
def color_generator():
    colors = [
        carla.Color(r=r, g=g, b=b) for r in [0, 255] for g in [0, 255] for b in [0, 255]
    ]
    while True:
        for color in colors:
            yield color

for i, color in zip(range(len(roads)), color_generator()):
    waypoint = roads[i][0]
    for j in range(1000):
        waypoint = waypoint.next(3)[0]
        draw_on_screen(world, waypoint.transform, color=color, life_time=10)
    time.sleep(0.5)

## 4. Repositioning vs controlling a vehicle

Two operations can look similar on screen but are fundamentally different.

- **Repositioning (`set_transform`)**: instant teleport to a new pose.
- **Controlling (`apply_control`)**: dynamic motion through throttle, steering, and brake.

Use repositioning for setup/debug. Use control commands for actual driving behavior.

In [20]:
vehicle = spawn_vehicle(world)
roads = world.get_map().get_topology()
waypoint = roads[0][0]

vehicle.set_transform(waypoint.transform)

A sequence of `set_transform` calls can create a movement-like visual effect, but it is still teleportation.
No physics-based driving logic is applied.

In [21]:
sleep_time = 1
waypoint_distance = 1

try:
    while True:
        waypoint = waypoint.next(waypoint_distance)[0]
        vehicle.set_transform(waypoint.transform)
        time.sleep(sleep_time)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

### Control a vehicle

In this section the vehicle is moved through `carla.VehicleControl`, which is the correct interface for driving.

Suggested workflow:
1. create small helpers (for example `move_forward`, `steer_left`);
2. apply one command at a time;
3. tick the world and sleep briefly to observe each action.

In [ ]:
vehicle = spawn_vehicle(world)

throttle = 0.6

def move_forward(vehicle, duration):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)
    start_time = time.time()
    while time.time() - start_time < duration:
        world.tick()
        time.sleep(0.1)

def steer_left(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    control.steer = -0.38
    vehicle.apply_control(control)
    world.tick()
    time.sleep(2)

try:
    move_forward(vehicle, 5)
    steer_left(vehicle)
    move_forward(vehicle, 20)
finally:
    vehicle.destroy()

### Example of random movement

This demo applies random throttle and steering values in a loop.
It is useful for testing control responsiveness, but it is not a stable navigation policy.

In [23]:
vehicle = spawn_vehicle(world)

def random_control(vehicle):
    control = carla.VehicleControl()
    control.throttle = random.uniform(0.5, 1.0)
    control.steer = random.uniform(-1.0, 1.0)
    vehicle.apply_control(control)

try:
    while True:
        random_control(vehicle)
        world.tick()
        time.sleep(0.1)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

## 5. Exercises (in class)

Use only functions and patterns already introduced in this notebook.

### Exercise 1: Multi-view spectator

Create three camera views and switch between them while the vehicle is moving:
- `1`: chase view (behind the car)
- `2`: top view (high above the car)
- `3`: side view (right side of the car)

Implementation hints:
1. Keep one variable `camera_mode`.
2. Update `camera_mode` with keyboard events.
3. In the main loop, call `move_spectator_to(...)` with different offsets based on the selected mode.
4. Keep autopilot enabled for the vehicle.

Success criteria:
- view changes immediately when keys are pressed;
- no flicker or camera jumps when switching;
- clean shutdown with `vehicle.destroy()` and `pygame.quit()`.

#### Solution

In [ ]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

pygame.init()
pygame.display.set_mode((420, 240))
pygame.display.set_caption("Exercise 1 - Multi-view spectator")

camera_mode = 1
running = True

try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_1:
                    camera_mode = 1
                elif event.key == pygame.K_2:
                    camera_mode = 2
                elif event.key == pygame.K_3:
                    camera_mode = 3

        transform = vehicle.get_transform()

        if camera_mode == 1:
            # Chase view.
            move_spectator_to(transform, spectator, distance=7.0, z=3.0, pitch=-15)
        elif camera_mode == 2:
            # Top view.
            move_spectator_to(transform, spectator, distance=0.0, z=35.0, pitch=-90)
        else:
            # Right-side view.
            move_spectator_to(transform, spectator, distance=0.0, y=8.0, z=3.0, yaw=90, pitch=-15)

        world.tick()
        time.sleep(0.02)
finally:
    vehicle.destroy()
    pygame.quit()

### Exercise 2: Safety ring around the car

Goal: draw a moving ring of debug markers around the vehicle.

Simplified approach:
1. At each loop, read `center = vehicle.get_transform().location`.
2. Create a fixed list of offsets (front, back, left, right, diagonals).
3. For each offset, draw one marker (`draw_point` or `draw_string`) at `center + offset`.
4. Use short `life_time` and redraw every tick.

Success criteria:
- markers stay centered on the vehicle while it moves;
- old markers fade quickly and the screen stays readable;
- ring size is easy to adjust by changing one variable.

#### Solution

In [ ]:
vehicle = spawn_vehicle(world, spawn_index=1)
vehicle.set_autopilot(True)

ring_radius = 3.0
ring_height = 0.6

offsets = [
    carla.Location( ring_radius, 0.0, ring_height),
    carla.Location(-ring_radius, 0.0, ring_height),
    carla.Location(0.0,  ring_radius, ring_height),
    carla.Location(0.0, -ring_radius, ring_height),
    carla.Location( ring_radius,  ring_radius, ring_height),
    carla.Location( ring_radius, -ring_radius, ring_height),
    carla.Location(-ring_radius,  ring_radius, ring_height),
    carla.Location(-ring_radius, -ring_radius, ring_height),
]

try:
    while True:
        center = vehicle.get_transform().location

        for offset in offsets:
            world.debug.draw_point(
                center + offset,
                size=0.08,
                color=carla.Color(r=0, g=255, b=255),
                life_time=0.2,
            )

        world.tick()
        time.sleep(0.03)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

### Exercise 3: Smooth manual driving

Build a basic manual controller with gradual commands:
- `w`: increase throttle step by step
- `s`: apply brake
- `a` / `d`: steer left/right
- no steering key: slowly return steer to `0`

Implementation hints:
1. Keep a `carla.VehicleControl()` object and update it every frame.
2. Clamp throttle and steer values to valid ranges.
3. Use a small update step (for example `0.02` to `0.05`) for smoother behavior.

Success criteria:
- vehicle starts and stops smoothly;
- steering is progressive, not abrupt;
- releasing keys leaves the car in a stable state.

#### Solution

In [ ]:
vehicle = spawn_vehicle(world, spawn_index=2)

pygame.init()
pygame.display.set_mode((420, 240))
pygame.display.set_caption("Exercise 3 - Smooth manual driving")

control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0)
throttle_step = 0.03
steer_step = 0.04
running = True


def clamp(value, low, high):
    return max(low, min(high, value))


try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                running = False

        keys = pygame.key.get_pressed()

        # Throttle and brake.
        if keys[pygame.K_w]:
            control.throttle = clamp(control.throttle + throttle_step, 0.0, 1.0)
            control.brake = 0.0
        elif keys[pygame.K_s]:
            control.throttle = clamp(control.throttle - throttle_step * 2, 0.0, 1.0)
            control.brake = clamp(control.brake + 0.08, 0.0, 1.0)
        else:
            control.throttle = clamp(control.throttle - throttle_step * 0.5, 0.0, 1.0)
            control.brake = 0.0

        # Progressive steering.
        if keys[pygame.K_a]:
            control.steer = clamp(control.steer - steer_step, -1.0, 1.0)
        elif keys[pygame.K_d]:
            control.steer = clamp(control.steer + steer_step, -1.0, 1.0)
        else:
            # Return to center smoothly.
            if control.steer > 0:
                control.steer = max(0.0, control.steer - steer_step)
            elif control.steer < 0:
                control.steer = min(0.0, control.steer + steer_step)

        vehicle.apply_control(control)
        move_spectator_to(vehicle.get_transform(), spectator, distance=7.0, z=3.0, pitch=-15)

        world.tick()
        time.sleep(0.03)
finally:
    vehicle.destroy()
    pygame.quit()

## 6. Optional home exercises

### Optional Exercise 1: Simple cruise control

Add a cruise mode toggled by `c`:
- when enabled, keep speed in a target range (for example 20-25 km/h);
- when disabled, return to manual controls.

Implementation hints:
1. Read vehicle speed from velocity magnitude.
2. If speed is below target, increase throttle slightly.
3. If speed is above target, reduce throttle or apply light brake.
4. Disable cruise automatically when `w` or `s` is pressed.

Success criteria:
- cruise keeps speed near target on straight roads;
- manual override works immediately;
- mode status is printed on screen or console.

#### Solution

In [ ]:
vehicle = spawn_vehicle(world, spawn_index=3)

pygame.init()
pygame.display.set_mode((440, 240))
pygame.display.set_caption("Optional 1 - Simple cruise control")

control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=0.0)
cruise_enabled = False
target_min_kmh = 20.0
target_max_kmh = 25.0
last_status_print = 0.0
running = True


def speed_kmh(actor):
    v = actor.get_velocity()
    return 3.6 * ((v.x * v.x + v.y * v.y + v.z * v.z) ** 0.5)


try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_c:
                    cruise_enabled = not cruise_enabled

        keys = pygame.key.get_pressed()

        # Manual override.
        if keys[pygame.K_w] or keys[pygame.K_s]:
            cruise_enabled = False

        # Basic manual steering always available.
        if keys[pygame.K_a]:
            control.steer = max(-1.0, control.steer - 0.04)
        elif keys[pygame.K_d]:
            control.steer = min(1.0, control.steer + 0.04)
        else:
            control.steer *= 0.8

        spd = speed_kmh(vehicle)

        if cruise_enabled:
            if spd < target_min_kmh:
                control.throttle = min(0.7, control.throttle + 0.03)
                control.brake = 0.0
            elif spd > target_max_kmh:
                control.throttle = max(0.0, control.throttle - 0.05)
                control.brake = min(0.3, control.brake + 0.06)
            else:
                control.throttle = max(0.22, min(control.throttle, 0.4))
                control.brake = 0.0
        else:
            # Manual throttle/brake when cruise is off.
            if keys[pygame.K_w]:
                control.throttle = min(1.0, control.throttle + 0.04)
                control.brake = 0.0
            elif keys[pygame.K_s]:
                control.throttle = max(0.0, control.throttle - 0.08)
                control.brake = min(1.0, control.brake + 0.08)
            else:
                control.throttle = max(0.0, control.throttle - 0.01)
                control.brake = 0.0

        vehicle.apply_control(control)
        move_spectator_to(vehicle.get_transform(), spectator, distance=8.0, z=3.0, pitch=-15)

        now = time.time()
        if now - last_status_print > 1.0:
            mode = "CRUISE" if cruise_enabled else "MANUAL"
            print(f"Mode: {mode} | Speed: {spd:5.1f} km/h | Throttle: {control.throttle:.2f} | Brake: {control.brake:.2f}")
            last_status_print = now

        world.tick()
        time.sleep(0.04)
finally:
    vehicle.destroy()
    pygame.quit()

### Optional Exercise 2: Short waypoint route

Create a short route of 20-30 waypoints ahead of the car and try to follow it.

Implementation hints:
1. Get current waypoint from vehicle location.
2. Build a list with repeated `next(distance)` calls.
3. Draw route waypoints with debug markers.
4. Move the car with simple steering toward the next target waypoint.

Success criteria:
- the route is visible and ordered;
- the car reaches most waypoints without teleporting;
- end condition stops the loop and cleans up actors.

#### Solution

In [ ]:
vehicle = spawn_vehicle(world, spawn_index=4)

# Build a short route of waypoints ahead.
start_wp = world.get_map().get_waypoint(vehicle.get_transform().location)
route = [start_wp]

for _ in range(29):
    next_candidates = route[-1].next(4.0)
    if not next_candidates:
        break
    route.append(random.choice(next_candidates))

print(f"Route length: {len(route)} waypoints")

# Draw route markers once.
for i, wp in enumerate(route):
    world.debug.draw_string(
        wp.transform.location + carla.Location(z=0.4),
        str(i),
        color=carla.Color(r=0, g=255, b=0),
        life_time=30.0,
    )

control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=0.0)
target_index = 1

try:
    while target_index < len(route):
        transform = vehicle.get_transform()
        location = transform.location
        forward = transform.get_forward_vector()

        target_loc = route[target_index].transform.location
        dx = target_loc.x - location.x
        dy = target_loc.y - location.y
        dist = (dx * dx + dy * dy) ** 0.5

        if dist < 2.2:
            target_index += 1
            continue

        ux = dx / max(dist, 1e-6)
        uy = dy / max(dist, 1e-6)

        # 2D cross product sign -> steering direction.
        cross = forward.x * uy - forward.y * ux
        steer_cmd = max(-0.6, min(0.6, cross * 2.5))

        control.steer = steer_cmd
        control.throttle = 0.32 if abs(steer_cmd) < 0.25 else 0.22
        control.brake = 0.0
        vehicle.apply_control(control)

        move_spectator_to(transform, spectator, distance=8.0, z=3.0, pitch=-15)
        world.debug.draw_point(
            target_loc + carla.Location(z=0.5),
            size=0.10,
            color=carla.Color(r=255, g=0, b=0),
            life_time=0.15,
        )

        world.tick()
        time.sleep(0.05)

    # End condition: stop the car.
    vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))
    world.tick()
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()